<a href="https://colab.research.google.com/github/Catherine-Nguyen88/project_voting/blob/main/Voting_Catherine's_Part.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Catherine's Part

# CLEANING, TRANSFORING, EDA
## merging datasets, data overview graphs, etc.

Taking Isabella's merging dataset code to run some EDA

In [1]:
! git clone https://github.com/Catherine-Nguyen88/project_voting

Cloning into 'project_voting'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 205 (delta 56), reused 45 (delta 18), pack-reused 90
Receiving objects: 100% (205/205), 65.68 MiB | 6.61 MiB/s, done.
Resolving deltas: 100% (92/92), done.
Updating files: 100% (66/66), done.


In [2]:
#Import statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.linear_model import LinearRegression

In [3]:
df = pd.read_csv('./project_voting/data/voting_VA.csv')
df

,Unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,11161,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,AL GORE,DEMOCRAT,5092,11925,20220315,TOTAL
1,11162,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6352,11925,20220315,TOTAL
2,11163,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,RALPH NADER,GREEN,220,11925,20220315,TOTAL
3,11164,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,OTHER,OTHER,261,11925,20220315,TOTAL
4,11165,2000,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,AL GORE,DEMOCRAT,16255,36846,20220315,TOTAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3731,71966,2020,VIRGINIA,VA,WINCHESTER CITY,51840,US PRESIDENT,OTHER,OTHER,24,12113,20220315,ELECTION DAY
3732,71967,2020,VIRGINIA,VA,WINCHESTER CITY,51840,US PRESIDENT,OTHER,OTHER,0,12113,20220315,PROVISIONAL
3733,71968,2020,VIRGINIA,VA,WINCHESTER CITY,51840,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,2825,12113,20220315,ABSENTEE
3734,71969,2020,VIRGINIA,VA,WINCHESTER CITY,51840,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,2370,12113,20220315,ELECTION DAY


In [4]:
# test demographic file
fname = './project_voting/data/county_data/0002_ds176_20105_county_E.csv'
dem = pd.read_csv(fname, encoding='latin-1')
dem.head()

# drop first row
dem = dem.drop([0])
dem.head()

# filter down: extract VA data only
dem_VA = dem[dem['STATE'] == 'Virginia']
print(dem_VA.shape) # the shape is correct for VA
dem_VA.head()

(0, 276)


,Unnamed: 0,X.2,X.1,X,GISJOIN,YEAR,STUSAB,REGIONA,DIVISIONA,STATE,...,JSDE003,JSDE004,JSDE005,JSDE006,JSDE007,JSDE008,JSDE009,JSDE010,JS5E001,JTIE001


In [5]:
a = dem_VA.dtypes
a

Unnamed: 0     int64
X.2            int64
X.1            int64
X             object
GISJOIN       object
               ...  
JSDE008       object
JSDE009       object
JSDE010       object
JS5E001       object
JTIE001       object
Length: 276, dtype: object

In [6]:
# modify the voting csv dataset to match the county names present in the demographics data
# change to title case and if the county doesnt end in "City", add "County"

def match_county_name(county_name):
  if county_name.endswith('CITY'):
    return county_name.title()
  else:
    return county_name.title() + ' County'

voting_df = df.copy()
voting_df['county_name'] = voting_df['county_name'].apply(lambda x: match_county_name(x))

voting_df.head()

,Unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,11161,2000,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,AL GORE,DEMOCRAT,5092,11925,20220315,TOTAL
1,11162,2000,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6352,11925,20220315,TOTAL
2,11163,2000,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,RALPH NADER,GREEN,220,11925,20220315,TOTAL
3,11164,2000,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,OTHER,OTHER,261,11925,20220315,TOTAL
4,11165,2000,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,AL GORE,DEMOCRAT,16255,36846,20220315,TOTAL


In [7]:
# now, for each county, get candidatevotes/totalvotes
voting_df['fractionalvotes'] = voting_df['candidatevotes']/voting_df['totalvotes']
voting_df.head()

,Unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode,fractionalvotes
0,11161,2000,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,AL GORE,DEMOCRAT,5092,11925,20220315,TOTAL,0.427002
1,11162,2000,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6352,11925,20220315,TOTAL,0.532662
2,11163,2000,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,RALPH NADER,GREEN,220,11925,20220315,TOTAL,0.018449
3,11164,2000,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,OTHER,OTHER,261,11925,20220315,TOTAL,0.021887
4,11165,2000,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,AL GORE,DEMOCRAT,16255,36846,20220315,TOTAL,0.441161


In [8]:
# i think the easiest option for our analysis so we have matched X (demographics) and Y (party) would be to add the demographics columns that we need to voting_df
# create merged dataframe
dem_VA = dem_VA.rename(columns={'COUNTY':'county_name'})
dem_VA.head()

,Unnamed: 0,X.2,X.1,X,GISJOIN,YEAR,STUSAB,REGIONA,DIVISIONA,STATE,...,JSDE003,JSDE004,JSDE005,JSDE006,JSDE007,JSDE008,JSDE009,JSDE010,JS5E001,JTIE001


In [9]:
# merge the datasets, dem_VA with voting_df
merged_df = voting_df.merge(dem_VA, on='county_name', how='left')
merged_df.head()

,Unnamed: 0_x,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,...,JSDE003,JSDE004,JSDE005,JSDE006,JSDE007,JSDE008,JSDE009,JSDE010,JS5E001,JTIE001
0,11161,2000,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,AL GORE,DEMOCRAT,5092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11162,2000,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6352,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11163,2000,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,RALPH NADER,GREEN,220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11164,2000,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,OTHER,OTHER,261,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11165,2000,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,AL GORE,DEMOCRAT,16255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Merges Estimate (E files) below:

In [10]:
voting_original = pd.read_csv('./project_voting/data/voting_VA.csv')
voting_original.head(5)

# modify voting CSV
def match_county_name(county_name):
  if county_name.endswith('CITY'):
    return county_name.title()
  else:
    return county_name.title() + ' County'

voting_df = voting_original.copy()
voting_df['county_name'] = voting_df['county_name'].apply(lambda x: match_county_name(x))
voting_df.head()

# now, for each county, get candidatevotes/totalvotes
voting_df['fractionalvotes'] = voting_df['candidatevotes']/voting_df['totalvotes']

# split voting data into separate years for merging
voting_2000 = voting_df[voting_df['year']==2000]
print(f'Years for voting_2000 {voting_2000["year"].unique()}')
voting_2004 = voting_df[voting_df['year']==2004]
print(f'Years for voting_2004 {voting_2004["year"].unique()}')
voting_2008 = voting_df[voting_df['year']==2008]
print(f'Years for voting_2008 {voting_2008["year"].unique()}')
voting_2012 = voting_df[voting_df['year']==2012]
print(f'Years for voting_2012 {voting_2012["year"].unique()}')
voting_2016 = voting_df[voting_df['year']==2016]
print(f'Years for voting_2016 {voting_2016["year"].unique()}')
voting_2020 = voting_df[voting_df['year']==2020]
print(f'Years for voting_2020 {voting_2020["year"].unique()}')

Years for voting_2000 [2000]
Years for voting_2004 [2004]
Years for voting_2008 [2008]
Years for voting_2012 [2012]
Years for voting_2016 [2016]
Years for voting_2020 [2020]


In [11]:
# 2000 Election
# extract demographic data
fname = './project_voting/data/county_data/0002_ds176_20105_county_E.csv'
dem = pd.read_csv(fname, encoding='latin-1')
# need to drop the 1st row
dem = dem.drop([0])
dem_VA = dem[dem['STATE'] == 'Virginia']
print(dem_VA.shape) # the shape is correct for VA
# rename for merging
dem_VA = dem_VA.rename(columns={'COUNTY':'county_name'})

# merge the datasets
merged_2000 = voting_2000.merge(dem_VA, on='county_name', how='left')
merged_2000.head(5)
# print(merged_2000.columns)

merged_2000['JSDE003']

(0, 276)


0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
531    NaN
532    NaN
533    NaN
534    NaN
535    NaN
Name: JSDE003, Length: 536, dtype: object

In [12]:
list(merged_2000.columns)
# important columns: year, state, state_po, county_name, county_fips, office, candidate, party, candidatevotes, totalvotes, fractionalvotes,

['Unnamed: 0_x',
 'year',
 'state',
 'state_po',
 'county_name',
 'county_fips',
 'office',
 'candidate',
 'party',
 'candidatevotes',
 'totalvotes',
 'version',
 'mode',
 'fractionalvotes',
 'Unnamed: 0_y',
 'X.2',
 'X.1',
 'X',
 'GISJOIN',
 'YEAR',
 'STUSAB',
 'REGIONA',
 'DIVISIONA',
 'STATE',
 'STATEA',
 'COUNTYA',
 'COUSUBA',
 'PLACEA',
 'TRACTA',
 'BLKGRPA',
 'CONCITA',
 'AIANHHA',
 'RES_ONLYA',
 'TRUSTA',
 'AIHHTLI',
 'AITSCEA',
 'ANRCA',
 'CBSAA',
 'CSAA',
 'METDIVA',
 'NECTAA',
 'CNECTAA',
 'NECTADIVA',
 'UAA',
 'CDCURRA',
 'SLDUA',
 'SLDLA',
 'SUBMCDA',
 'SDELMA',
 'SDSECA',
 'SDUNIA',
 'PCI',
 'PUMAA',
 'GEOID',
 'NAME',
 'BTTRA',
 'BTBGA',
 'JLZE001',
 'JLZE002',
 'JLZE003',
 'JLZE004',
 'JLZE005',
 'JLZE006',
 'JLZE007',
 'JLZE008',
 'JLZE009',
 'JLZE010',
 'JLZE011',
 'JLZE012',
 'JLZE013',
 'JLZE014',
 'JLZE015',
 'JLZE016',
 'JLZE017',
 'JLZE018',
 'JLZE019',
 'JLZE020',
 'JLZE021',
 'JLZE022',
 'JLZE023',
 'JLZE024',
 'JLZE025',
 'JLZE026',
 'JLZE027',
 'JLZE028',
 'JL

In [13]:
# important columns:

In [14]:
# 2004 Election
# extract demographic data
fname = './project_voting/data/county_data/0002_ds176_20105_county_E.csv'
dem = pd.read_csv(fname, encoding='latin-1')
# need to drop the 1st row
dem = dem.drop([0])
dem_VA = dem[dem['STATE'] == 'Virginia']
print(dem_VA.shape) # the shape is correct for VA
# rename for merging
dem_VA = dem_VA.rename(columns={'COUNTY':'county_name'})

# merge the datasets
merged_2004 = voting_2004.merge(dem_VA, on='county_name', how='left')
merged_2004.head(5)

(0, 276)


,Unnamed: 0_x,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,...,JSDE003,JSDE004,JSDE005,JSDE006,JSDE007,JSDE008,JSDE009,JSDE010,JS5E001,JTIE001
0,20838,2004,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,JOHN KERRY,DEMOCRAT,5518,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20839,2004,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,7726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20840,2004,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,OTHER,OTHER,112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20841,2004,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,JOHN KERRY,DEMOCRAT,22088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20842,2004,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,21189,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# 2008 election
# extract demographic data
fname = './project_voting/data/county_data/0002_ds191_20125_county_E.csv'
dem = pd.read_csv(fname, encoding='latin-1')
# need to drop the 1st row
dem = dem.drop([0])
dem_VA = dem[dem['STATE'] == 'Virginia']
print(dem_VA.shape) # the shape is correct for VA
# rename for merging
dem_VA = dem_VA.rename(columns={'COUNTY':'county_name'})

# merge the datasets
merged_2008 = voting_2008.merge(dem_VA, on='county_name', how='left')
merged_2008.head(5)

(134, 191)


,Unnamed: 0_x,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,...,QXSE007,QX6E001,QX7E001,QX7E002,QX7E003,QX8E001,QX8E002,QX8E003,QZTE001,QZ6E001
0,30189,2008,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,BARACK OBAMA,DEMOCRAT,7607,...,10973,21017,21017,14286,6731,14286,10070,4216,741,153800
1,30190,2008,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,JOHN MCCAIN,REPUBLICAN,7833,...,10973,21017,21017,14286,6731,14286,10070,4216,741,153800
2,30191,2008,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,OTHER,OTHER,183,...,10973,21017,21017,14286,6731,14286,10070,4216,741,153800
3,30192,2008,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,BARACK OBAMA,DEMOCRAT,29792,...,30576,42332,42332,37549,4783,37549,24648,12901,1080,332400
4,30193,2008,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,JOHN MCCAIN,REPUBLICAN,20576,...,30576,42332,42332,37549,4783,37549,24648,12901,1080,332400


In [16]:
# 2012 Election
# extract demographic data
fname = './project_voting/data/county_data/0002_ds206_20145_county_E.csv'
dem = pd.read_csv(fname, encoding='latin-1')
# need to drop the 1st row
dem = dem.drop([0])
dem_VA = dem[dem['STATE'] == 'Virginia']
print(dem_VA.shape) # the shape is correct for VA
# rename for merging
dem_VA = dem_VA.rename(columns={'COUNTY':'county_name'})

# merge the datasets
merged_2012 = voting_2012.merge(dem_VA, on='county_name', how='left')
merged_2012.head(5)

(133, 192)


,Unnamed: 0_x,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,...,ABGVE001,ABGWE001,ABGWE002,ABGWE003,ABGXE001,ABGXE002,ABGXE003,ABIHE001,ABIOE001,ABITE001
0,39540,2012,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,BARACK OBAMA,DEMOCRAT,7655,...,21054,21054,14289,6765,14289,10053,4236,715,26.2,152500
1,39541,2012,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,MITT ROMNEY,REPUBLICAN,8213,...,21054,21054,14289,6765,14289,10053,4236,715,26.2,152500
2,39542,2012,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,OTHER,OTHER,183,...,21054,21054,14289,6765,14289,10053,4236,715,26.2,152500
3,39543,2012,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,BARACK OBAMA,DEMOCRAT,29757,...,43128,43128,38537,4591,38537,25135,13402,1115,30.0,317300
4,39544,2012,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,MITT ROMNEY,REPUBLICAN,23297,...,43128,43128,38537,4591,38537,25135,13402,1115,30.0,317300


In [17]:
# 2016 Election
# extract demographic data
fname = './project_voting/data/county_data/0002_ds225_20165_county_E.csv'
dem = pd.read_csv(fname, encoding='latin-1')
# need to drop the 1st row
dem = dem.drop([0])
dem_VA = dem[dem['STATE'] == 'Virginia']
print(dem_VA.shape) # the shape is correct for VA
# rename for merging
dem_VA = dem_VA.rename(columns={'COUNTY':'county_name'})

# merge the datasets
merged_2016 = voting_2016.merge(dem_VA, on='county_name', how='left')
merged_2016.head(5)

(133, 187)


,Unnamed: 0_x,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,...,AF67E003,AF67E004,AF67E005,AF67E006,AF67E007,AF7PE001,AF7PE002,AF7PE003,AF89E001,AF9LE001
0,48891,2016,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,HILLARY CLINTON,DEMOCRAT,6740,...,15206,14137,1069,149,11575,13819,9605,4214,749,151900
1,48892,2016,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,DONALD TRUMP,REPUBLICAN,8583,...,15206,14137,1069,149,11575,13819,9605,4214,749,151900
2,48893,2016,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,OTHER,OTHER,495,...,15206,14137,1069,149,11575,13819,9605,4214,749,151900
3,48894,2016,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,HILLARY CLINTON,DEMOCRAT,33345,...,51563,49890,1673,354,33063,39431,25584,13847,1156,317300
4,48895,2016,VIRGINIA,VA,Albemarle County,51003,US PRESIDENT,DONALD TRUMP,REPUBLICAN,19259,...,51563,49890,1673,354,33063,39431,25584,13847,1156,317300


In [18]:
# 2020 election
# extract demographic data
fname = './project_voting/data/county_data/0002_ds249_20205_county_E.csv'
dem = pd.read_csv(fname, encoding='latin-1')
# need to drop the 1st row
dem = dem.drop([0])
dem_VA = dem[dem['STATE'] == 'Virginia']
print(dem_VA.shape) # the shape is correct for VA
# rename for merging
dem_VA = dem_VA.rename(columns={'COUNTY':'county_name'})

# merge the datasets
merged_2020 = voting_2020.merge(dem_VA, on='county_name', how='left')
merged_2020.head(5)

<ipython-input-18-6b3b079ca811>:4: DtypeWarning: Columns (4,5,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,26

(133, 993)


,Unnamed: 0_x,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,...,AMWSE004,AMWSE005,AMWSE006,AMWSE007,AMWSE008,AMWSE009,AMWSE010,AMWSE011,AMWSE012,AMWSE013
0,70379,2020,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,5495,...,9929,8310,2429,6248,1300,1444,102,10,414,3225
1,70380,2020,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,2072,...,9929,8310,2429,6248,1300,1444,102,10,414,3225
2,70381,2020,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,11,...,9929,8310,2429,6248,1300,1444,102,10,414,3225
3,70382,2020,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,JO JORGENSEN,LIBERTARIAN,66,...,9929,8310,2429,6248,1300,1444,102,10,414,3225
4,70383,2020,VIRGINIA,VA,Accomack County,51001,US PRESIDENT,JO JORGENSEN,LIBERTARIAN,122,...,9929,8310,2429,6248,1300,1444,102,10,414,3225


## Merge all election years

In [19]:
#Combining all year voting data
dfs = [merged_2000, merged_2004, merged_2008, merged_2012, merged_2016, merged_2020]

#Finding common columns
com_cols = set.intersection(*[set(df.columns) for df in dfs])

#Same columns
common = [df[list(com_cols)] for df in dfs]

#Combining along row
VA_estimates0 = pd.concat(common, ignore_index=True)
print(VA_estimates0.shape)
VA_estimates0.head()

#Counting number of NA's
na_counts = VA_estimates0.isna().sum()
print(na_counts)

(3736, 54)
Unnamed: 0_y       1780
CSAA               3736
REGIONA            3736
candidatevotes        0
NECTAA             3736
Unnamed: 0_x          0
SDELMA             3736
AITSCEA            3736
GISJOIN            1780
state                 0
RES_ONLYA          3736
SLDUA              3736
CDCURRA            3736
totalvotes            0
TRUSTA             3736
TRACTA             3736
SDUNIA             3736
SLDLA              3736
COUNTYA            1780
NECTADIVA          3736
DIVISIONA          3736
AIHHTLI            3736
YEAR               1780
county_fips           0
party                 0
BLKGRPA            3736
PUMAA              3736
PLACEA             3736
COUSUBA            3736
year                  0
ANRCA              3736
UAA                3736
CNECTAA            3736
office                0
SDSECA             3736
state_po              0
STATEA             1780
candidate             0
BTBGA              3736
version               0
STUSAB             1780
fract

In [20]:
#Dropping columns where all values are NA's
VA_estimates = VA_estimates0.dropna(axis=1, how='all')
print(VA_estimates.shape)

VA_estimates.head()

#Counting number of NA's
#na_counts = VA_voting1.isna().sum()
#print(na_counts)

(3736, 23)


,Unnamed: 0_y,candidatevotes,Unnamed: 0_x,GISJOIN,state,totalvotes,COUNTYA,YEAR,county_fips,party,...,STATEA,candidate,version,STUSAB,fractionalvotes,STATE,GEOID,mode,NAME,county_name
0,NaN,5092,11161,NaN,VIRGINIA,11925,NaN,NaN,51001,DEMOCRAT,...,NaN,AL GORE,20220315,NaN,0.427002,NaN,NaN,TOTAL,NaN,Accomack County
1,NaN,6352,11162,NaN,VIRGINIA,11925,NaN,NaN,51001,REPUBLICAN,...,NaN,GEORGE W. BUSH,20220315,NaN,0.532662,NaN,NaN,TOTAL,NaN,Accomack County
2,NaN,220,11163,NaN,VIRGINIA,11925,NaN,NaN,51001,GREEN,...,NaN,RALPH NADER,20220315,NaN,0.018449,NaN,NaN,TOTAL,NaN,Accomack County
3,NaN,261,11164,NaN,VIRGINIA,11925,NaN,NaN,51001,OTHER,...,NaN,OTHER,20220315,NaN,0.021887,NaN,NaN,TOTAL,NaN,Accomack County
4,NaN,16255,11165,NaN,VIRGINIA,36846,NaN,NaN,51003,DEMOCRAT,...,NaN,AL GORE,20220315,NaN,0.441161,NaN,NaN,TOTAL,NaN,Albemarle County


Change datatypes of each off the columns as they are all objects right now:

In [21]:
dtype1 = dem_VA.dtypes
dtype1

Unnamed: 0     int64
GISJOIN       object
YEAR          object
STUSAB        object
REGIONA       object
               ...  
AMWSE009      object
AMWSE010      object
AMWSE011      object
AMWSE012      object
AMWSE013      object
Length: 993, dtype: object

In [22]:
import pandas as pd

# Create a sample DataFrame
data = {'A': [1, 2, 3], 'B': ['4', '5', '6']}
df = pd.DataFrame(data)

# Before casting
print("Before casting:")
print(df.dtypes)

# Cast column 'B' to integer
df['B'] = df['B'].astype(int)

# After casting
print("\nAfter casting:")
print(df.dtypes)


Before casting:
A     int64
B    object
dtype: object

After casting:
A    int64
B    int64
dtype: object


# EDA, cleaning and transforming of merged data


In [25]:
import pandas as pd
df = pd.read_csv('/content/project_voting/voting_est.csv')
df.head()

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,...,Female: 50 to 54 years,Female: 55 to 59 years,Female: 60 and 61 years,Female: 62 to 64 years,Female: 65 and 66 years,Female: 67 to 69 years,Female: 70 to 74 years,Female: 75 to 79 years,Female: 80 to 84 years,Female: 85 years and over
0,2008,VIRGINIA,VA,Accomack,51001,US PRESIDENT,BARACK OBAMA,DEMOCRAT,7607,15623,...,1446.0,1316.0,727.0,638.0,369.0,690.0,806.0,791.0,441.0,412.0
1,2008,VIRGINIA,VA,Accomack,51001,US PRESIDENT,JOHN MCCAIN,REPUBLICAN,7833,15623,...,1446.0,1316.0,727.0,638.0,369.0,690.0,806.0,791.0,441.0,412.0
2,2008,VIRGINIA,VA,Accomack,51001,US PRESIDENT,OTHER,OTHER,183,15623,...,1446.0,1316.0,727.0,638.0,369.0,690.0,806.0,791.0,441.0,412.0
3,2008,VIRGINIA,VA,Albemarle,51003,US PRESIDENT,BARACK OBAMA,DEMOCRAT,29792,50984,...,3889.0,3840.0,1251.0,1468.0,1038.0,1161.0,1884.0,1546.0,1470.0,1269.0
4,2008,VIRGINIA,VA,Albemarle,51003,US PRESIDENT,JOHN MCCAIN,REPUBLICAN,20576,50984,...,3889.0,3840.0,1251.0,1468.0,1038.0,1161.0,1884.0,1546.0,1470.0,1269.0


In [26]:
# look at column names to see what we have narrowed down too
df.columns

Index(['year', 'state', 'state_po', 'county_name', 'county_fips', 'office',
       'candidate', 'party', 'candidatevotes', 'totalvotes',
       'Not Hispanic or Latino: White alone',
       'Not Hispanic or Latino: Black or African American alone',
       'Not Hispanic or Latino: American Indian and Alaska Native alone',
       'Not Hispanic or Latino: Asian alone',
       'Not Hispanic or Latino: Native Hawaiian and Other Pacific Islander alone',
       'Not Hispanic or Latino: Two or more races', 'Hispanic or Latino',
       'No schooling completed', 'Regular high school diploma',
       'GED or alternative credential', 'Some college, less than 1 year',
       'Some college, 1 or more years, no degree', 'Associate's degree',
       'Bachelor's degree', 'Master's degree', 'Professional school degree',
       'Doctorate degree', 'Under .50', '.50 to .99', '1.00 to 1.24',
       '1.25 to 1.49', '1.50 to 1.84', '1.85 to 1.99', '2.00 and over',
       'Male: 18 and 19 years', 'Male: 20 ye

In [ ]:
# we want to find net vote count at the county level, {Total Republican}-{Total Democrat}

# make a new column called net_vote_count
df['net_vote_count'] =

# TREES

In [23]:
# Process data:
y = df['Price']
#y =  np.log(df['Price']) # Compare with and without logs
vars = ['Beds','Number Of Reviews']
X = pd.concat([df.loc[:,vars], type_dummy, nbhd_dummy, room_dummy],axis=1)

# Train-test split:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=100)

# Fit decision tree:
cart = DecisionTreeRegressor(max_depth = 3) # Create a regression object
cart = cart.fit(X_train, y_train) # Fit the regression
var_names = cart.feature_names_in_

# Visualize results:
plot_tree(cart,filled=True,feature_names=var_names)
plt.show()

## Make Predictions on the Test Set
y_hat = cart.predict(X_test)

## Residuals, Rsq, RMSE
e = y_test - y_hat
sns.kdeplot(e) # Notice the residuals aren't centered around zero
plt.show()
rsq = 1-np.sum(e**2)/np.sum( (y_test - np.mean(y_train))**2)
RMSE = np.sqrt(np.mean(e**2))

print('Rsq: ', rsq, '\n')
print('RMSE: ', RMSE, '\n')

KeyError: 'Price'

In [ ]:
county.head()